In [1]:
# first, lets get the imports we need 
from typing import List, Tuple, Dict, Any, Set
from collections import defaultdict
from itertools import product, chain
from tqdm import tqdm
import torch
from auto_circuit.data import load_datasets_from_json
from auto_circuit.experiment_utils import load_tl_model
from auto_circuit.prune_algos.mask_gradient import mask_gradient_prune_scores
from auto_circuit.prune_algos.edge_attribution_patching import edge_attribution_patching_prune_scores
from auto_circuit.data import BatchKey, PromptDataLoader
from auto_circuit.types import AblationType, PatchType, PruneScores, CircuitOutputs
from auto_circuit.utils.ablation_activations import src_ablations, batch_src_ablations
from auto_circuit.utils.graph_utils import patch_mode, patchable_model, set_all_masks, train_mask_mode
from auto_circuit.utils.tensor_ops import batch_avg_answer_diff
from auto_circuit.utils.misc import repo_path_to_abs_path
from auto_circuit.visualize import draw_seq_graph
from auto_circuit.types import Edge, Node



In [2]:
device = "cpu" #TODO: debug mps error
model = load_tl_model("gpt2-small", device)

/Users/oliverdaniels-koch/Library/Caches/pypoetry/virtualenvs/elk-experiments-AZ2LBS3Q-py3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer


In [3]:
factorized=True
slice_output="last_seq"
separate_qkv=True
seq_len=None

In [4]:
from auto_circuit.model_utils.transformer_lens_utils import factorized_dest_nodes, factorized_src_nodes

In [5]:
model = patchable_model(
    model, 
    factorized=factorized, 
    slice_output=slice_output, 
    separate_qkv=separate_qkv, 
    seq_len=seq_len,
    resid_src=False, 
    resid_dest=False,
    attn_src=True,
    attn_dest=True,
    mlp_src=True,
    mlp_dest=True,
)

In [7]:
path = repo_path_to_abs_path("datasets/ioi/ioi_vanilla_template_prompts.json")
train_loader, test_loader = load_datasets_from_json(
    model=model,
    path=path,
    device=device,
    prepend_bos=True,
    batch_size=16,
    train_test_size=(128, 128),
)

In [8]:
attribution_scores: PruneScores = mask_gradient_prune_scores(
    model=model,
    dataloader=train_loader,
    official_edges=None,
    grad_function="logit",
    answer_function="avg_diff",
    mask_val=0.0,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
